In [ ]:
!kill -9 $(lsof -t -i:8188)

In [ ]:
 %pip install mpmath==1.3.0 nest_asyncio  numpy simpleeval color_matcher google.generativeai

In [ ]:
%pip uninstall xformers -y

In [ ]:
%pip install -U open_clip_torch --force-reinstall

In [ ]:
%cd /notebooks/ComfyUI
import nest_asyncio
nest_asyncio.apply()



%run main.py  --disable-xformers  

In [ ]:
%cd /notebooks/ComfyUI
!/root/miniconda3/bin/conda run -n sd_ui_f_conda python  main.py  --disable-xformers  

In [ ]:
!/root/miniconda3/envs/sd_ui_f_conda/bin/python3 main.py  &